# Berechnung der BERT-Embeddings

Im ersten Schritt mit den Sprachmodellen wirst du nun die Embeddings selbst ausrechnen.

Du lernst dabei schon kennen, wie du ein Modell benutzt und ein bisschen auch den Umgang mit `PyTorch`.

## Daten einladen

Wie gewohnt lädst du die Daten ein. Die Sprachmodelle nutzen einen eingebauten Tokenisierer, der auch Teilworte erkennt. Daher nützen dir die linguistisch voranalysierten Daten nicht sehr viel und du kannst einfach die Ursprungsdaten verwenden:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT id, url, title, datePublished FROM articles \
                  WHERE datePublished<'2021-01-01' \
                  ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

Die Sprachmodelle bauen eigentlich alle auf `PyTorch` auf. `PyTorch` ist eine Bibliothek für tiefe neuronale Netze und funktioniert am besten mit einer Grafikkarten (die in diesem Notebook noch nicht unbedingt erforderlich ist):

In [ ]:
!pip install torch
!pip install pytorch_pretrained_bert

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Es stehen verschiedene Sprachmodelle bereit, `bert-base-multilingual-uncased` arbeitet nur mit Kleinbuchstaben uns ist sprachunabhängig. Zunächst erzeugst du einen `BertTokenizer`: 

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Nun kannst du auch das dazu korrespondierende Modell instanziieren:

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

Die folgende Funktion dient dazu, Texte mit BERT zu tokeniseren und anschließend deren Embeddings auszurechnen:

In [ ]:
def embed_text(text):
    # Text mit Start- und Endmarkern ergänzen
    marked_text = "[CLS] " + str(text) + " [SEP]"
    # Nur die ersten 512 Tokens verwenden
    tokenized = tokenizer.tokenize(marked_text)[0:512]
    # Und in IDs wandeln
    indexed = tokenizer.convert_tokens_to_ids(tokenized)
    # Der Einfachheit halber packst du alles in einen "Satz"
    segments_ids = [1] * len(tokenized)
    # Bisher hast du mit Listen gearbeitet, diese wandelst du jetzt in Tensoren
    tokens_tensor = torch.tensor([indexed])
    segments_tensors = torch.tensor([segments_ids])
    # Nun überträgst du die Token- und Segment-Tensoren in das Modell
    with torch.no_grad():
        # als Ergebnis erhältst du alle (!) Embedding-Layer
        el, _ = model(tokens_tensor, segments_tensors)
    return el

Für jeden einzelnen Titel kannst du nun die Embedding-Layer ausrechnen. Den kontextualisierten Embedding-Vektor findest du im letzten Layer und mittelst diesen. Später wirst du nur den Embedding-Vektor des ersten, *vollkontextualisierten* Tokens verwenden (nämlich von `[CLS]`). Da die Berechnung ein bisschen dauert, nutzt du `tqdm` zur Forschrittsanzeigen:

In [ ]:
from tqdm.auto import tqdm
tokenized_texts = []

sentence_embeddings = []

for text in tqdm(df["title"], total=len(df)):
    el = embed_text(text)
    token_vecs = el[11][0]
    embedding = torch.mean(token_vecs, dim=0)
    sentence_embeddings.append(embedding.numpy())

Die Embeddings packst du jetzt in einen `DataFrame` und speicherst sie in der Datenbank ab.

In [ ]:
text_bert = pd.DataFrame(sentence_embeddings)
text_bert.index = df.index

text_bert.to_sql("bert_articles", sql, index_label="id", if_exists="replace")

Du kannst nun die Embedding-Layer für einen neuen Text ausrechnen (den du erfunden hast):

In [ ]:
et = embed_text("Microsoft schafft Umsatzsprung")

Genau wie oben bestimmst du den Mittelwert der Embeddings aller Tokens des letzten Layers:

In [ ]:
token_vecs = et[11][0]
embedding = torch.mean(token_vecs, dim=0)

Mit der Cosinus-Ähnlichkeit von `scikit-learn` kannst du die (kontextualisierten) Ähnlichkeiten zu allen anderen Dokumenten berechnen:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
r = cosine_similarity(sentence_embeddings, [embedding.numpy()])

Mithilfe von `argmax` findest du den Index des Dokuments, der diesem am ähnlichsten ist:

In [ ]:
r.argmax()

In [ ]:
df.iloc[990]

## Embeddings bilden kontextualisierte Ähnlichkeiten ab

Du hast gesehen, wie sich mithilfe von Sprachmodellen kontextualisierte Embeddings berechnen lassen. Der Prozess ist etwas komplizierter als bei den "einfachen" Word Embeddings und funktioniert grundsätzlich nur mit Sätzen oder längeren Entitäten.

Kontextualisierte Ähnlichkeiten zu finden funktioniert allerdings nicht völlig überzeugend, das hat mit der semantischen Transformation viel besser geklappt. Darauf sind die BERT-Modelle auch nicht abgestimmt, sondern eher auf das sog. *Finetuning*, das du anschließend kennenlernen wirst.